# Features : Analysing Results XLMR

In [64]:
import pandas as pd
import numpy as np
import seaborn as sns
from statistics import mean, variance

1) Read in results:

In [65]:
# from Jowita: results = pd.DataFrame(columns=['FEATURE','EXPERIMENT','FINE_TUNING','f1_1', 'f1_2', 'f1_3','f1_4'])

feat_results_raw = pd.read_csv('features_gathering_results_XLMR.csv', sep=',')
feat_results_raw.head()

,FEATURE,DATASET,EXPERIMENT,FINE_TUNING,eval_loss,f1 score
0,Feature 1,1,1,fine_tuning 1,0.338931,0.822798
1,Feature 1,1,1,fine_tuning 2,0.330252,0.825857
2,Feature 1,1,2,fine_tuning 1,0.270912,0.788478
3,Feature 1,1,2,fine_tuning 2,0.231738,0.821153
4,Feature 1,2,1,fine_tuning 1,0.311741,0.820181


In [66]:
feat_results_raw['FEATURE'] = feat_results_raw['FEATURE'].apply(lambda x: str(x).split(" ")[1])

In [67]:
feat_results_raw['FEATURE'] = feat_results_raw['FEATURE'].apply(lambda x: int(x))
feat_results_raw.sort_values(by=['FEATURE']).reset_index()

,index,FEATURE,DATASET,EXPERIMENT,FINE_TUNING,eval_loss,f1 score
0,0,1,1,1,fine_tuning 1,0.338931,0.822798
1,15,1,4,2,fine_tuning 2,0.279347,0.734716
2,14,1,4,2,fine_tuning 1,0.295434,0.751245
3,13,1,4,1,fine_tuning 2,0.346349,0.823551
4,12,1,4,1,fine_tuning 1,0.335090,0.802797
5,10,1,3,2,fine_tuning 1,0.343239,0.739414
6,9,1,3,1,fine_tuning 2,0.319708,0.824901
7,8,1,3,1,fine_tuning 1,0.347899,0.813867
8,11,1,3,2,fine_tuning 2,0.263628,0.796426
9,6,1,2,2,fine_tuning 1,0.286912,0.762887


In [68]:
#Transforming it to the format feat_results_raw[['FEATURE','EXPERIMENT','FINE_TUNING','f1_1', 'f1_2', 'f1_3','f1_4']]
feat_results_transf = pd.DataFrame(columns=['FEATURE','EXPERIMENT','FINE_TUNING','f1_1', 'f1_2', 'f1_3','f1_4'])
list_of_Series = []

for feat in feat_results_raw['FEATURE'].unique().tolist():
    
    fine_tuning1 = feat_results_raw[(feat_results_raw['FEATURE'] == feat) & (feat_results_raw['FINE_TUNING'] == 'fine_tuning 1')]
    fine_tuning1_exp1 = fine_tuning1[fine_tuning1['EXPERIMENT'] == 1].reset_index().sort_values(by='DATASET')
    fine_tuning1_exp2 = fine_tuning1[fine_tuning1['EXPERIMENT'] == 2].reset_index().sort_values(by='DATASET')
    
    fine_tuning2 = feat_results_raw[(feat_results_raw['FEATURE'] == feat) & (feat_results_raw['FINE_TUNING'] == 'fine_tuning 2')]
    fine_tuning2_exp1 = fine_tuning2[fine_tuning2['EXPERIMENT'] == 1].reset_index().sort_values(by='DATASET')
    fine_tuning2_exp2 = fine_tuning2[fine_tuning2['EXPERIMENT'] == 2].reset_index().sort_values(by='DATASET')
    
    Series_fine_tuning1_exp1 = pd.Series([fine_tuning1_exp1['FEATURE'][0], fine_tuning1_exp1['EXPERIMENT'][0], fine_tuning1_exp1['FINE_TUNING'].iloc[0], fine_tuning1_exp1['f1 score'].iloc[0], fine_tuning1_exp1['f1 score'].iloc[1], fine_tuning1_exp1['f1 score'].iloc[2], fine_tuning1_exp1['f1 score'].iloc[3]], index=feat_results_transf.columns)
    Series_fine_tuning1_exp2 = pd.Series([fine_tuning1_exp2['FEATURE'][0], fine_tuning1_exp2['EXPERIMENT'][0], fine_tuning1_exp2['FINE_TUNING'].iloc[0], fine_tuning1_exp2['f1 score'].iloc[0], fine_tuning1_exp2['f1 score'].iloc[1], fine_tuning1_exp2['f1 score'].iloc[2], fine_tuning1_exp2['f1 score'].iloc[3]], index=feat_results_transf.columns)
    Series_fine_tuning2_exp1 = pd.Series([fine_tuning2_exp1['FEATURE'][0], fine_tuning2_exp1['EXPERIMENT'][0], fine_tuning2_exp1['FINE_TUNING'].iloc[0], fine_tuning2_exp1['f1 score'].iloc[0], fine_tuning2_exp1['f1 score'].iloc[1], fine_tuning2_exp1['f1 score'].iloc[2], fine_tuning2_exp1['f1 score'].iloc[3]], index=feat_results_transf.columns)
    Series_fine_tuning2_exp2 = pd.Series([fine_tuning2_exp2['FEATURE'][0], fine_tuning2_exp2['EXPERIMENT'][0], fine_tuning2_exp2['FINE_TUNING'].iloc[0], fine_tuning2_exp2['f1 score'].iloc[0], fine_tuning2_exp2['f1 score'].iloc[1], fine_tuning2_exp2['f1 score'].iloc[2], fine_tuning2_exp2['f1 score'].iloc[3]], index=feat_results_transf.columns)
                             
        
    list_of_Series += [Series_fine_tuning1_exp1, Series_fine_tuning1_exp2, Series_fine_tuning2_exp1, Series_fine_tuning2_exp2]

feat_results_transf = feat_results_transf.append(list_of_Series, ignore_index=True).sort_values(by=['FEATURE','EXPERIMENT','FINE_TUNING']).reset_index()
feat_results_transf.head()

,index,FEATURE,EXPERIMENT,FINE_TUNING,f1_1,f1_2,f1_3,f1_4
0,0,1,1,fine_tuning 1,0.822798,0.820181,0.813867,0.802797
1,2,1,1,fine_tuning 2,0.825857,0.814429,0.824901,0.823551
2,1,1,2,fine_tuning 1,0.788478,0.762887,0.739414,0.751245
3,3,1,2,fine_tuning 2,0.821153,0.821162,0.796426,0.734716
4,4,2,1,fine_tuning 1,0.774135,0.771580,0.791872,0.784773


2) Data Cleaning: Filter for f1 scores + mean/variance/std calculation of f1 score

In [69]:
feat_results_f1 = feat_results_transf[['FEATURE','EXPERIMENT', 'FINE_TUNING', 'f1_1','f1_2','f1_3','f1_4']]
    
feat_results_f1 = feat_results_f1.reindex(columns = ['FEATURE','EXPERIMENT', 'FINE_TUNING','f1_1','f1_2','f1_3','f1_4','mean','variance', 'standard dev'])

# changing to procent format
feat_results_f1['f1_1'] = feat_results_f1['f1_1'].apply(lambda x: x*100)
feat_results_f1['f1_2'] = feat_results_f1['f1_2'].apply(lambda x: x*100)
feat_results_f1['f1_3'] = feat_results_f1['f1_3'].apply(lambda x: x*100)
feat_results_f1['f1_4'] = feat_results_f1['f1_4'].apply(lambda x: x*100)

for index, row in feat_results_f1.iterrows():
    
    if row['f1_4'] == 0:
        feat_results_f1.at[index,'mean'] = np.mean([row['f1_1'],row['f1_2'],row['f1_3']])
        feat_results_f1.at[index,'variance'] = np.var([row['f1_1'],row['f1_2'],row['f1_3']])
        feat_results_f1.at[index,'standard dev'] = np.std([row['f1_1'],row['f1_2'],row['f1_3']])
    else:
        feat_results_f1.at[index,'mean'] = np.mean([row['f1_1'],row['f1_2'],row['f1_3'],row['f1_4']])
        feat_results_f1.at[index,'variance'] = np.var([row['f1_1'],row['f1_2'],row['f1_3'],row['f1_4']])
        feat_results_f1.at[index,'standard dev'] = np.std([row['f1_1'],row['f1_2'],row['f1_3'],row['f1_4']])

feat_results_f1.to_csv('Features_all_results_XLMR.csv', index=False)

3) Change format to: [EXPERIMENT, FINE_TUNING_1_f1_score_mean, FINE_TUNING_1_f1_score_std, FINE_TUNING_2_f1_score_mean, FINE_TUNING_2_f1_score_std, relative improvement]

Note: for each feature two experiments changing eval/fine-tune TRUE group

In [70]:
feat_results_compressed = pd.DataFrame(columns=['FEATURE','EXPERIM','FINT1_f1_mean','FINT1_f1_std','FINT2_f1_mean','FINT2_f1_std','rel. Diff'])

list_of_Series = []

for index, row in feat_results_f1.iterrows():
    
    if (index % 2 == 0):
            
        f2_mean = feat_results_f1.iloc[index+1]['mean']
        f2_std = feat_results_f1.iloc[index+1]['standard dev']
        rel_diff = ((f2_mean - row['mean'])/ row['mean'])*100
            
            
        list_of_Series += [pd.Series([row['FEATURE'],row['EXPERIMENT'], row['mean'], row['standard dev'], f2_mean, f2_std, rel_diff], index=feat_results_compressed.columns)]

feat_results_compressed = feat_results_compressed.append(list_of_Series, ignore_index=True)

feat_results_compressed.to_csv('Features_compressed_XLMR.csv', index=False)
feat_results_compressed.to_excel("features_results_analysed_XLMR.xlsx") 